In [14]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json

In [15]:
urlbase = "https://data.cityofnewyork.us/resource/"

In [16]:
def desde_fecha(fecha_str):
       return f'{fecha_str}T00:00:00.000'

In [17]:
def hasta_fecha(fecha_str):
   return f'{fecha_str}T23:59:59.000'

In [18]:
def extraccion_actual(ini, fin, token):
    url_eventos = f"{urlbase}tvpp-9vvx.json"
    
    param = {
        "$where": f"start_date_time >= '{ini}' AND start_date_time <= '{fin}'",    
    }

    header = {"X-App-Token": token}
    eventos = requests.get(url=url_eventos, params=param, headers=header)
    

    if eventos.status_code != 200:
        print(f"Error Parques: {eventos.text}")
 
    assert eventos.status_code == 200, "Error en la extracción de eventos"
    return eventos.json()




In [19]:
token = os.getenv('NYC_OPEN_DATA_TOKEN')
assert token is not None, "Falta la variable de entorno NYC_OPEN_DATA_TOKEN"


fecha_hoy_str = datetime.now().strftime('%Y-%m-%d')
fecha_actual = desde_fecha(fecha_hoy_str)
fecha_fin = hasta_fecha(fecha_hoy_str)


json_eventos = extraccion_actual(fecha_actual, fecha_fin, token)

In [20]:
df = pd.DataFrame(json_eventos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   event_id             149 non-null    object
 1   event_name           148 non-null    object
 2   start_date_time      149 non-null    object
 3   end_date_time        149 non-null    object
 4   event_agency         149 non-null    object
 5   event_type           149 non-null    object
 6   event_borough        149 non-null    object
 7   event_location       149 non-null    object
 8   street_closure_type  149 non-null    object
 9   community_board      149 non-null    object
 10  police_precinct      149 non-null    object
 11  cemsid               146 non-null    object
 12  event_street_side    3 non-null      object
dtypes: object(13)
memory usage: 15.3+ KB


In [21]:

df['start_date_time'] = pd.to_datetime(df['start_date_time'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
df["start_date_time"] = df["start_date_time"].dt.strftime('%H:%M:%S')
df['end_date_time'] = pd.to_datetime(df['end_date_time'], errors='coerce',  format='%Y-%m-%dT%H:%M:%S.%f')
df["end_date_time"] = df["end_date_time"].dt.strftime('%H:%M:%S')



In [22]:
df.columns
df = df.drop(["event_id", "event_agency", "street_closure_type", 'community_board','police_precinct', 'cemsid',
       'event_street_side' ], axis = 1)


In [23]:

riesgo_map = {
    'Parade': 10,
    'Athletic Race / Tour': 10,
    'Street Event': 8,
    'Special Event': 7,
    'Plaza Event': 6,
    'Plaza Partner Event': 6,
    'Theater Load in and Load Outs': 5,
    'Religious Event': 3,
    'Farmers Market': 2,
    'Sidewalk Sale': 2,
    'Production Event': 1,
    'Sport - Adult': 1,
    'Sport - Youth': 1,
    'Miscellaneous': 1,
    'Open Street Partner Event': 2
}


df['nivel_riesgo_tipo'] = df['event_type'].map(riesgo_map)

In [24]:
df = df.sort_values(by = "nivel_riesgo_tipo", ascending= False)
df

,event_name,start_date_time,end_date_time,event_type,event_borough,event_location,nivel_riesgo_tipo
0,construction,00:00:00,12:59:00,Special Event,Manhattan,Fort Tryon Park: Billings Lawn,7
1,Lawn Closure - Levin Playground Lawn,00:00:00,23:59:00,Special Event,Manhattan,Central Park: Lawn Outside Levin Playground (S...,7
2,Maintenance,00:00:00,23:59:00,Special Event,Manhattan,Fort Tryon Park: Billings Lawn,7
3,Lawn Closure - Pilgrim Hill,00:00:00,23:59:00,Special Event,Manhattan,Central Park: Pilgrim Hill,7
4,Lawn Closure - East 102 St. Landscape A B,00:00:00,23:59:00,Special Event,Manhattan,"Central Park: East 102nd St Landscape A ,Cen...",7
...,...,...,...,...,...,...,...
144,Soccer - Non Regulation,20:00:00,22:00:00,Sport - Adult,Brooklyn,Bushwick Inlet Park: Soccer-01,1
145,Soccer - Non Regulation,20:00:00,22:00:00,Sport - Adult,Queens,Astoria Park: Soccer-01,1
146,Soccer - Non Regulation,20:30:00,22:00:00,Sport - Youth,Staten Island,Blood Root Valley: Greenbelt Recreation Center...,1
147,Soccer - Non Regulation,20:30:00,22:00:00,Sport - Adult,Brooklyn,Bush Terminal Park: Soccer-02,1
